In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

#    Previsão de *Default* em Financiamento Bancário

## Por aluna matrícula 1931133087 

O presente trabalho utiliza o dataset Home Equity DAtaset(HMEQ) disponível no Kaggle para, mediante modelo de Machine Learning, prever o Default ou não do financiamento bancário contratado pelo cliente.

O Dataset HMEQ apresenta informações de 5960 contratantes e de seus financiamentos bancários distribuídas nas 12 variáveis abaixo e também a variável resposta "BAD" que informa se o cliente deixou de honrar seu empréstimo (default) ou não.

As variáveis preditoras constantes da base de dados são:

* LOAN: Valor da parcela (valor $);

* MORTDUE: Valor devido na hipoteca existente (valor $);

* VALUE: Valor corrente da propriedade financiada (valor $);

* REASON: Destinação do crédito (se debt consolidatio= DebtCon ou home improvement= HomeImp;

* JOB: Ocupação. Possui sex classes: Other, ProfExe, Office, Mgr, Self e Sales;

* YOJ: Anos no emprego atual (em anos);

* DEROG: Número de relatórios depreciativos;

* DELINQ: Número de linhas de crédito inadimplentes;

* CLAGE: Idade da linha de crédito mais antiga;

* NINQ: Número de linhas de crédito recentes;

* CLNO: Número de linhas de crédito;

* DEBTIC : Razão Dívida/Rendimento.

A variável resposta BAD é binária e apresenta as seguintes classes: 0 - o cliente não ficou inadimplente e 1 - o cliente apresentou Default. Logo, é esta variável que conterá as previsões realizadas.


# 1. Carregamento da Base de Dados HMEQ

In [ ]:
# Carregamento do Dataset:
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df.shape

A Base de Dados possui 5960 observações e 13 colunas

In [ ]:
# Visualizando os dados:
df.head()

In [ ]:
# Verificando uma amostra aleatória
df.sample(5)

In [ ]:
# Verificando os tipos dos dados e os tamanhos
df.info()

Nesta visualização, verifica-se grande quantidade de colunas com dados 'missing'. Aliás, somente a coluna com a variável resposta 'BAD e a 'VALUE' não demandam tratamento.
Além disso, há duas variáveis do tipo 'object', ou seja, não numéricas. 
Logo, o tratamento dos dados se destinará a solucionar estas duas situações para que seja viável a aplicação do modelo escolhido.

# 2. Tratamento dos Dados

## 2.1. Tratamento dos dados Faltantes

### 2.1.1. Exclusão de observações com informações sobre o financiamento nulas.

Estudando os dados, entendeu-se que as observações onde as variáveis 'MORTDUE' (valor devido) e 'VALUE' (valor da propriedade) estão com dados nulos (em conjunto), devem ser excluídas pois não agregam informações relevantes para o estudo de predição, podendo até distorcer, caso haja imputação.

Desta forma, inicialmente será feito a verificação das observações com esta característica e após, sua exclusão.

In [ ]:
# Verificando quantas observações possuem esta característica: 
df[df['MORTDUE'].isnull() & df['VALUE'].isnull()]

Serão excluídos os 27 registros que possuem esta caracteristica:

In [ ]:
# Usando o comando drop para excluir os registros que possuem valores nulos nas 2 varíaveis com dados 
#sobre o atual financiamento.
df.dropna(how='all', subset = ['MORTDUE', 'VALUE'], inplace=True)

In [ ]:
# conferindo se o comando de exclusão deu certo: 
df[df['MORTDUE'].isnull() & df['VALUE'].isnull()]

### 2.1.2 Imputação de dados faltantes



#### VARIÁVEIS MORTDUE E VALUE
Ainda existem muitos casos de valores nulos nas colunas 'MORTDUE' e 'VALUE'. Para imputar os valores destas colunas adotaremos o seguinte:
Caso de nulo no valor da propriedade (VALUE): Será imputado o valor da dívida atual (MORTDUE), garantindo assim que o valor da propriedade será ao menos o valor da dívida.
Caso de nulo no valor da dívida autal (MORTDUE): Será imputado o valor da propriedade garantidora da operação, considerando assim o valor da propriedade como valor estimado do financiamento.

In [ ]:
# verificando a quantidade de casos enquadrados nesta situação (nulo em MORTDUE ou VALUE:
df[df['MORTDUE'].isnull() | df['VALUE'].isnull()]

Aqui se verifica que quase 10% da base de dados encontra-se nesta situação (576 registros).

In [ ]:
# imputando o valor da Propriedade pelo valor da dívida:
df['VALUE'] = df.apply(lambda row: row['MORTDUE'] if np.isnan(row['VALUE']) else row['VALUE'],axis=1)

In [ ]:
# imputando o valor da dívida pelo valor da propriedade: 
df['MORTDUE'] = df.apply(lambda row: row['VALUE'] if np.isnan(row['MORTDUE']) else row['MORTDUE'],axis=1)

In [ ]:
# Conferindo que não existem mais registros com valores nulos nas colunas MORTDUE e VALUE:
df[df['MORTDUE'].isnull() | df['VALUE'].isnull()]

In [ ]:
# Verificando os primeiros dados após efeito da imputação
df.head(10)

In [ ]:
# Verificando como ficou o Dataset após esta primeira imputação:
df.info()

A base de dados ficou com 27 registros a menos, com nenhum valor nulo nas quatro primeiras colunas.

VARIÁVEL 'REASON'

In [ ]:
# Verificando a quantidade de dados nulos:
df[df['REASON'].isnull()]

Verifica-se 242 com dado nulo na variável 'REASON'.

In [ ]:
# Verificando as classes da variável 'REASON' e a frequência de cada um delas.
df['REASON'].value_counts()

Aqui verifica-se que a maior frequência é da classe 'DebtCon'.

Como apoio à decisão sobre imputar os dados missing da variável 'REASON' pelo valor da classe com maior frequência, decidiu-se verificar em uma tabela cruzada se as classes desta variável apresentam grande diferença de proporção na variável resposta. Um desproporção indicaria tentar outro método para imputação visto a possibilidade de distorção na variável resposta.  

In [ ]:
# Verificando as proporção dos dados distribuídos entre as variáveis 'REASON' e 'BAD':
totals=pd.crosstab(df['REASON'],df['BAD'],margins=True).reset_index()
percentages = pd.crosstab(df['REASON'],
   df['BAD']).apply(lambda row: row/row.sum(),axis=1).reset_index()
totals



In [ ]:
# Verificando os percentuais cruzados entre 'REASON' e 'BAD':
percentages

A análise da tabela cruzada mostrou pouco desbalanceamento entre classes da variável resposta (0,18 e 0,21), então optou-se pela imputação usando a classe de maior frequência, 'DebtCon', para o tratamento dos dados missing.

In [ ]:
# Realizando a imputação da variável 'REASON':
df['REASON'].fillna('DebtCon', inplace=True)


In [ ]:
# Verificando como ficou a distribuição de frequências da variável 'REASON':
df['REASON'].value_counts()

In [ ]:
# Verificando como ficou o Dataset:
df.info()

In [ ]:
# Verificando as classes e frequência da variável 'JOB':
df['JOB'].value_counts()

A classe 'Other' possui a maioria dos registros válidos.

In [ ]:
# Verificando os registros nulos na variável 'JOB':
df[df['JOB'].isnull()]

Verifica-se 267 registros com dado faltante na variável 'JOB'.

Visto a classe 'Other' possuir a maior parte dos registros, optou-se pela imputação dos valores 'missing' pelo valor desta classe.

In [ ]:
# Imputação dos valores nulos da variável 'JOB'pelo valor da classe 'Other':
df['JOB'].fillna('Other', inplace=True)

In [ ]:
# Verificando como ficou a distribuição das classes da variável 'JOB':
df['JOB'].value_counts()

In [ ]:
#Verificando como ficou o Dataset:
df.info()

VARIÁVEL 'YOJ'

In [ ]:
# Verificando a distribuição da variável:
df['YOJ'].value_counts()

In [ ]:
# Verificando os dados nulos da variável 'YOJ':
df[df['YOJ'].isnull()]

Constata-se 505 registros com dados nulos na variável 'YOJ';
Visto a variável ser contínua, verificou-se a existência de vários domínios, sem predominância de um deles. Desta forma partiu-se para a análise do seu resumo estatístico e de sua distribuição:

In [ ]:
# Verificando o resumo de medidas estísiticas da variável 'YOJ':
df['YOJ'].describe()

In [ ]:
# Verificando uma medida estatística extra, a mediana:
df['YOJ'].median()

In [ ]:
# Verificando a distribuição dos dados por meio do Histograma:
df['YOJ'].plot.hist(bins=50)

Considerando os dados estarem mais distribuídos à esquerda, usaremos a Mediana ao invés da Média para a imputação dos valores missing da coluna "YOJ". 

In [ ]:
# Efetuando a imputação pela Mediana: 
df['YOJ'].fillna(7, inplace=True)

In [ ]:
# Verificando o dataset:
df.info()

VARIÁVEIS 'DEROG', 'DELINQ, 'CLAGE', 'NINQ', 'CLNO', DEBTINC':

In [ ]:
# Verificando as classes e distribuição da 'DEROG':
df['DEROG'].value_counts()

In [ ]:
# Verificando a quantidade de registros com dados missing na variável 'DEROG': 
df[df['DEROG'].isnull()]

Tendo em vista a quase totalidade dos registros estarem com valor zero para a variável 'DEROG' usaremos a imputação por zero.

In [ ]:
# Verificando as classes e distribuição da 'DELINQ':
df['DELINQ'].value_counts()

In [ ]:
# Verificando a quantidade de registros com dados missing na variável 'DELINQ': 
df[df['DELINQ'].isnull()]

Tendo em vista, assim como a 'DEROG, que a quase totalidade dos registros estão com valor zero para a variável 'DELINQ' usaremos a imputação por zero

In [ ]:
# Verificando as classes e frequência da variável 'CLAGE':
df['CLAGE'].value_counts()

In [ ]:
df[df['CLAGE'].isnull()]

Aqui também trata-se de variável contínua sem predominância de classe. É necessária a análise de suas medidas estatísticas para a tomada de decisão quanto a imputação de dados nulos.

In [ ]:
# Verificando o resumo de medidas estísiticas da variável 'CLAGE':
df['CLAGE'].describe()

In [ ]:
# Verificando a Mediana:
df['CLAGE'].median()

In [ ]:
# Verificando a distribuição dos dados por meio do Histograma:
df['CLAGE'].plot.hist(bins=50)

Tendo em vista a distribuição dos dados, para CLAGE usaremos a imputação pela mediana

In [ ]:
# Verificando as classes e distribuição entre estas para a variável 'NINQ': 
df['NINQ'].value_counts()

In [ ]:
# Verificando os dados nulos para a variável 'NINQ':
df[df['NINQ'].isnull()]

In [ ]:
# Sumário estatístico para 'NINQ':
df['NINQ'].describe()

In [ ]:
# Verificando a Mediana:
df['NINQ'].median()

In [ ]:
# Verificando o Histograma:
df['CLAGE'].plot.hist(bins=50)

Para NINQ usaremos a mediana, visto os dados estarem fortemente distribuidos à esquerda da curva.

In [ ]:
# Verificando classes e frequência para variável 'CLNO':
df['CLNO'].value_counts()

In [ ]:
# Verificando os valores Nulos: 
df[df['CLNO'].isnull()]

In [ ]:
# Verificando as medidas estatísticas:
df['CLNO'].describe()

In [ ]:
# Verificando a Mediana:
df['CLNO'].median()

In [ ]:
# Verificando o Histograma:
df['CLNO'].plot.hist(bins=50)

Para CLNO usaremos a mediana. Concentração maior à esquerda.

In [ ]:
# Verificando as classes e distribuição da variável 'DEBTINC':
df['DEBTINC'].value_counts()

A variável DEBTINC é contínua sem classes predominante.

In [ ]:
# Verificando os valores Nulos: 
df[df['DEBTINC'].isnull()]

In [ ]:
# Verificando o resumo estatístico:
df['DEBTINC'].describe()

In [ ]:
# Verificando a Mediana: 
df['DEBTINC'].median()

In [ ]:
# Verificando o histograma: 
df['DEBTINC'].plot.hist(bins=50)

Visto os dados não estarem muito dispersos, para a variável DEBTINC usaremos a média, como critério para a imputação dos dados faltantes.

Finalmente, após o término da análise das seis últimas variáveis, procederemos a imputação conforme a seguir: 
DEROG: por zero;
DELINQ: por zero;
CLAGE: pela Mediana;
NINQ: pela Mediana;
CLNO: pela Mediana:
DEBTINC: pela Média.

In [ ]:
# Imputação dos dados nulos:
df['DEROG'].fillna(0, inplace=True)
df['DELINQ'].fillna(0, inplace=True)
df['CLAGE'].fillna(173.48, inplace=True)
df['NINQ'].fillna(1, inplace=True)
df['CLNO'].fillna(20, inplace=True)
df['DEBTINC'].fillna(33.79, inplace=True)

In [ ]:
# Verificando o dataset após todo o tratamento de Dados 'Missing's':
df.info()

## 2.2. ANÁLISE EXPLORATÓRIA DAS VARIÁVEIS EXPLICATIVAS E SUAS RELAÇÕES COM A VARIÁVEL RESPOSTA 'BAD'

### 2.2.1. Verificando graficamente a Distribuição da Variável Resposta 'BAD': 

In [ ]:
# Importando 
import seaborn as sns


In [ ]:
# Verificando inicialmente a Tabela Cruzada

y = df['BAD'].astype(object) 
count = pd.crosstab(index = y, columns="count")
percentage = pd.crosstab(index = y, columns="frequency")/pd.crosstab(index = y, columns="frequency").sum()
pd.concat([count, percentage], axis=1)

In [ ]:
# Plotando o gráfico da Frequência da Variável Resposta 'BAD'
ax = sns.countplot(x=y, data=df).set_title("Distribuição da Variável Resposta 'BAD'")

Pode-se perceber que o índice de inadimplência ('default') é de 19,70%

### 2.2.2. Verificando graficamente a relação da variável resposta 'BAD' e algumas variáveis explicativas

'BAD' versus 'JOB'



In [ ]:
# Plotando o Gráfico de Barras Empilhadas mostrando a relação entre a variável 'BAD' e "JOB":
JOB=pd.crosstab(df['JOB'],df['BAD'])
JOB.div(JOB.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='JOB x BAD', figsize=(4,4))

No gráfico acima é confrontada a inadimplência (variável 'BAD') com o tipo de ocupação ('JOB'). Verifica-se que o segmento voltado à Vendas (Sales) possui o maior índice de inadimplência, cerca de 30%, sendo que o menor índice de default foi da classe 'Oficce'.

'BAD' versus 'REASON': 



In [ ]:
REASON=pd.crosstab(df['REASON'],df['BAD'])
REASON.div(REASON.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='REASON x BAD', figsize=(4,4))

O gráfico acima mostra a relação entre a inadimplência e a finalidade do crédito. Verifica-se que a inadimplência é ligeiramente menor para a finalidade "DebtConf".


'BAD' versus 'MORTDUE'



In [ ]:
sns.stripplot(x='BAD', y='MORTDUE', data=df, linewidth=1)

O gráfico acima mostra a relação entre a inadimplência e o valor do financiamento. Pode-se constatar que com a exceção de 4 outlies com valores elevados de dívida, não existe grande diferença no valor médio do crédito do cliente insolvente para aquele que conseguiu honrá-lo;



'BAD'versus 'DEBTINC':

In [ ]:
sns.stripplot(x='BAD', y='DEBTINC', data=df, linewidth=1)

Este gráfico relaciona a variável 'BAD' com a 'DEBTINC'. A variável 'DEBTINC recebe a razão entre o valor dos débitos e os rendimentos do financiado, ou seja, quanto maior o seu valor, maior é endividamento do cliente. Graficamente percebe-se que nenhum clientes na classe que dos que conseguiram pagar seu empréstimo possuem DEBTINC maior que 50. Por outro lado, é grande a quantidade de inadimplentes com valores elevados nesta variável. Portanto, provavelmente esta variável terá grande impacto no modelo de predição. 

## 2.3. Tranformando as variáveis categóricas em 'dummies'

A base de dados conta com duas variáveis do tipo 'object' e que deverão ser tratadas para que se possa utilizá-las no modelo de predição: "REASON" e "JOB". 
A opção escolhida foi a transformação em 'dummies, conforme é efetuado a seguir:

In [ ]:
# Verificando as variáveis do dataset:
df.info()

In [ ]:
# Transformando as variáveis 'REASON' e 'JOB' em 'dummies':
df = pd.get_dummies(df, columns=['REASON', 'JOB'])

In [ ]:
# Verificando como ficou o dataset após o processo de transformações em 'dummies':
df.info()

Conforme verifica-se acima, o dataset final, tratado e pronto para ser utilizado no modelo, contará com 19 variáveis, todas numéricas. 

In [ ]:
# Visualizando o dataset após o processo de 'dummies':
df.head().T

# 3. Verificação das Correlações entre as variáveis

Previamente à aplicação dos modelos de Machine Learning escolhidos para a predição, é importante verificar a colinearidade entre as variáveis explicativas, para que, com a devida exclusão das variáveis correlacionadas, podermos evitar o overfitting do modelo. É o que será realizado neste tópico. 

In [ ]:
df.corr()

In [ ]:
# Importando o pacote matplotlib
import matplotlib.pyplot as plt

In [ ]:
#Create Correlation matrix
corr = df.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(10,8))
#Generate Color Map
colormap = sns.diverging_palette(220, 10, as_cmap=True)
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

Analisando o Gráfico da Matriz de Correlação, verifica-se que as Variáveis 'MORTDUE' e 'VALUE' possuem forte correlação positiva (0,89). Para evitar distorçoes no modelo, optou-se pela exclusão de uma delas: a 'VALUE'.

Da mesma forma, será excluída da análise da variável dummificada REASON_HomeImp.

# 4. APLICAÇÃO DO MODELO


A fim de conseguirmos previsões acerca da possibilidade de 'default' ou não a partir dos dados do dataset, vamos utilizar dois algoritmos diferentes de árvores de decisão: Random Forest e o XGBoost, ambos modelos de Machine Learning. 

## 4.1. Aplicação do Modelo de Random Forest

In [ ]:
# Importando o pacote do Sklearn:
from sklearn.model_selection import train_test_split

In [ ]:
# Separando os dados de Treino, Validação e Teste, usando a proporção 80/20:
train, test = train_test_split(df, test_size=0.20, random_state=42)
train, valid = train_test_split(train, test_size=0.20, random_state=42)
train.shape, valid.shape, test.shape

In [ ]:
# Definindo colunas de entrada. Excluiremos a Variável Resposta e as duas com colinearidade:
feats = [c for c in df.columns if c not in ['BAD', 'VALUE', 'REASON_HomeImp']]
feats

In [ ]:
# Importando o pacote RandomForestClassifier necessário para rodar o modelo:

from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Instanciando o modelo com 200 árvores de decisão
rf = RandomForestClassifier(n_estimators=200, random_state=42) 

In [ ]:
# Treinando o Modelo: 
rf.fit(train[feats], train['BAD'])

In [ ]:
# Fazendo as previsões para os dados de Validação:
preds_val= rf.predict(valid[feats])
preds_val

In [ ]:
# Importando o pacote necessário para verificarmos a acurácio do modelo

from sklearn.metrics import accuracy_score

In [ ]:
# Verificando a predição nos dados de Validação:
accuracy_score(valid['BAD'], preds_val)

Encontramos uma acurácia de 0,93 nos dados de validação.

APURANDO A ACURÁCIA NOS DADOS DE TESTE:

In [ ]:
# Verificando a acurácia do modelo nos dados de teste:
preds_test = rf.predict(test[feats])
accuracy_score(test['BAD'], preds_test)

A Acurácia encontrada nos dados de teste foi de 0,915754.

VARIÁVEIS MAIS IMPORTANTES PARA O MODELO

In [ ]:
# avaliando a importância de cada coluna (cada variável de entrada)

pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

Este gráfico demonstra-se a importância de cada uma das variáveis para a predição realizada. Verifica-se que a variável que teve mais peso no modelo foi a DEBTINC, seguida de MORTDUE, DELINQ e LOAN.

MATRIZ DE CONFUSÃO

In [ ]:
#importando a biblioteca necessária para plotar o gráfico de Matriz de Confusão

import scikitplot as skplt

In [ ]:
# Gerando a Matriz de Confusão 
skplt.metrics.plot_confusion_matrix(valid['BAD'], preds_val)

Na diagonal principal da Matriz de Confusão encontram-se as predições corretas realizadas pelo modelo aplicado. Verifica-se 760 acertos de não evento, ou seja, aqueles onde o modelo acertou que não haveria o Default e 126 acertos de evento, quando o modelo acertou a incorrência do default. 

## 4.2. Aplicação do Modelo XGBoost com Cross Validation

In [ ]:
# Fazendo uma cópia do Dataset para a aplicação do outro modelo. Chamaremos de df1: 
df1 = df.copy()

In [ ]:
# Separando o dataframe em dados de treino e Teste. Não será apartado dados para validação visto que usaremos
# a validação cruzada onde faremos várias validações com dados aleatórios do dataset de treino.

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
train, test = train_test_split(df1, test_size=0.20, random_state=42)

# Não vamos mais usar o dataset de validação

train.shape, test.shape

In [ ]:
# definindo colunas de entrada

feats = [c for c in df1.columns if c not in ['BAD', 'VALUE', 'REASON_HomeImp']]

APLICAÇÃO DO XGBoost: Árvores encadeadas

In [ ]:
# Importar o modelo
from xgboost import XGBClassifier

# Instanciar o modelo
xgb = XGBClassifier(n_estimators=200, n_jobs=-1, random_state=42, learning_rate=0.05)

In [ ]:
# Usando o Cross validation

from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb, train[feats], train['BAD'], n_jobs=-1, cv=5) # estimator= xgb

# Definiremos 5 splits para realizar a validação cruzada:
scores, scores.mean() 

O array acima mostra a acurácia obtida em cada split de validação. O último valor é a média encontrada: 0,912

In [ ]:
# Usando o XGB para treinamento e predição 

xgb.fit(train[feats], train['BAD'])

In [ ]:
# Fazendo predições
preds = xgb.predict(test[feats])

In [ ]:
# Medir o desempenho do modelo
from sklearn.metrics import accuracy_score

accuracy_score(test['BAD'], preds)

Comparando o modelo de Random Forest e o XGBoost para os dados estudados, verifica-se que o primeiro possui leve vantagem em termos de acurácia: 0,9157 contra 0,9115

VARIÁVEIS MAIS IMPORTANTES PARA O MODELO XGBoost

In [ ]:
pd.Series(xgb.feature_importances_, index=feats).sort_values().plot.barh()

Com o algoritmo XGBoost percebemos diferenças quanto à importância de cada uma das variáveis para a predição realizada. Verifica-se que a variável que teve mais peso no modelo também foi a DEBTINC, porém no XGBoost esta vem seguida DELINQ e DEROG.

# 5.CONCLUSÃO

Com a realização do presente estudo, verificou-se que, após a análise e tratamento realizados nos dados originais e aplicação do modelo de Machine Learnig Random Forest (com os parâmetros utilizados acima) foi possível encontrar uma acurácia na predição nos dados de teste de 0,9154, ligeiramente melhor do que o modelo XGBoost usando com validação cruzada. Desta forma, para os dados estudados, o modelo escolhido foi o Random Forest.